<a href="https://colab.research.google.com/github/benayas1/ALV_Framework/blob/master/cornell_birds_training_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Import apex for mixed precision training
#!cd /kaggle/input/apexpytorch/ && pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./ >> /dev/null

#!wget https://github.com/benayas1/benatools/archive/master.zip -P bena
#!unzip bena/master.zip

!pip install benatools >> /dev/null

# Install resnest
#!pip install ../input/resnest50-fast-package/resnest-0.0.6b20200701/resnest/ >> /dev/null
!pip install soundfile

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path
import typing as tp
import cv2
import librosa
import random
import audioread
import soundfile as sf
import os
import time as time
import matplotlib.pyplot as plt

import benatools as bena
from benatools.tools import MultiStratifiedKFold
from benatools.tf.efficient_net import create_efn
import efficientnet.tfkeras as efn


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

import tensorflow as tf
import tensorflow.keras.backend as K

!pip install gcsfs
import gcsfs
from google.colab import auth
from google.cloud import storage


#from apex import amp

#import resnest.torch as resnest_torch

from sklearn.metrics import f1_score
from tqdm import tqdm
import glob

COLAB = True
GS = False
if GS:
    os.environ.setdefault("GCLOUD_PROJECT", "omega-cosmos-116215")
    GS_BUCKET = 'benayas_kaggle'
    !echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
    !curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
    !sudo apt-get -y -q update
    !sudo apt-get -y -q install gcsfuse
    auth.authenticate_user()
    !mkdir -p data
    !gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 {GS_BUCKET} data

Seeding everything

In [3]:
# Seed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    tf.random.set_seed(seed)
    
seed_everything(42)

Setting up TPU's 

In [4]:
DEVICE = "TPU"

if DEVICE == "TPU":
    print("connecting to TPU...")
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        print("Could not connect to TPU")
        tpu = None

    if tpu:
        try:
            print("initializing  TPU ...")
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.TPUStrategy(tpu)
            print("TPU initialized")
        except _:
            print("failed to initialize TPU")
    else:
        DEVICE = "GPU"

if DEVICE != "TPU":
    print("Using default strategy for CPU and single GPU")
    strategy = tf.distribute.get_strategy()

if DEVICE == "GPU":
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
    

AUTO     = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

connecting to TPU...
Running on TPU  grpc://10.91.190.234:8470
initializing  TPU ...
INFO:tensorflow:Initializing the TPU system: grpc://10.91.190.234:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.91.190.234:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


TPU initialized
REPLICAS: 8


In [5]:
# Coding the bird names

BIRD_CODE = {
    'aldfly': 0, 'ameavo': 1, 'amebit': 2, 'amecro': 3, 'amegfi': 4,
    'amekes': 5, 'amepip': 6, 'amered': 7, 'amerob': 8, 'amewig': 9,
    'amewoo': 10, 'amtspa': 11, 'annhum': 12, 'astfly': 13, 'baisan': 14,
    'baleag': 15, 'balori': 16, 'banswa': 17, 'barswa': 18, 'bawwar': 19,
    'belkin1': 20, 'belspa2': 21, 'bewwre': 22, 'bkbcuc': 23, 'bkbmag1': 24,
    'bkbwar': 25, 'bkcchi': 26, 'bkchum': 27, 'bkhgro': 28, 'bkpwar': 29,
    'bktspa': 30, 'blkpho': 31, 'blugrb1': 32, 'blujay': 33, 'bnhcow': 34,
    'boboli': 35, 'bongul': 36, 'brdowl': 37, 'brebla': 38, 'brespa': 39,
    'brncre': 40, 'brnthr': 41, 'brthum': 42, 'brwhaw': 43, 'btbwar': 44,
    'btnwar': 45, 'btywar': 46, 'buffle': 47, 'buggna': 48, 'buhvir': 49,
    'bulori': 50, 'bushti': 51, 'buwtea': 52, 'buwwar': 53, 'cacwre': 54,
    'calgul': 55, 'calqua': 56, 'camwar': 57, 'cangoo': 58, 'canwar': 59,
    'canwre': 60, 'carwre': 61, 'casfin': 62, 'caster1': 63, 'casvir': 64,
    'cedwax': 65, 'chispa': 66, 'chiswi': 67, 'chswar': 68, 'chukar': 69,
    'clanut': 70, 'cliswa': 71, 'comgol': 72, 'comgra': 73, 'comloo': 74,
    'commer': 75, 'comnig': 76, 'comrav': 77, 'comred': 78, 'comter': 79,
    'comyel': 80, 'coohaw': 81, 'coshum': 82, 'cowscj1': 83, 'daejun': 84,
    'doccor': 85, 'dowwoo': 86, 'dusfly': 87, 'eargre': 88, 'easblu': 89,
    'easkin': 90, 'easmea': 91, 'easpho': 92, 'eastow': 93, 'eawpew': 94,
    'eucdov': 95, 'eursta': 96, 'evegro': 97, 'fiespa': 98, 'fiscro': 99,
    'foxspa': 100, 'gadwal': 101, 'gcrfin': 102, 'gnttow': 103, 'gnwtea': 104,
    'gockin': 105, 'gocspa': 106, 'goleag': 107, 'grbher3': 108, 'grcfly': 109,
    'greegr': 110, 'greroa': 111, 'greyel': 112, 'grhowl': 113, 'grnher': 114,
    'grtgra': 115, 'grycat': 116, 'gryfly': 117, 'haiwoo': 118, 'hamfly': 119,
    'hergul': 120, 'herthr': 121, 'hoomer': 122, 'hoowar': 123, 'horgre': 124,
    'horlar': 125, 'houfin': 126, 'houspa': 127, 'houwre': 128, 'indbun': 129,
    'juntit1': 130, 'killde': 131, 'labwoo': 132, 'larspa': 133, 'lazbun': 134,
    'leabit': 135, 'leafly': 136, 'leasan': 137, 'lecthr': 138, 'lesgol': 139,
    'lesnig': 140, 'lesyel': 141, 'lewwoo': 142, 'linspa': 143, 'lobcur': 144,
    'lobdow': 145, 'logshr': 146, 'lotduc': 147, 'louwat': 148, 'macwar': 149,
    'magwar': 150, 'mallar3': 151, 'marwre': 152, 'merlin': 153, 'moublu': 154,
    'mouchi': 155, 'moudov': 156, 'norcar': 157, 'norfli': 158, 'norhar2': 159,
    'normoc': 160, 'norpar': 161, 'norpin': 162, 'norsho': 163, 'norwat': 164,
    'nrwswa': 165, 'nutwoo': 166, 'olsfly': 167, 'orcwar': 168, 'osprey': 169,
    'ovenbi1': 170, 'palwar': 171, 'pasfly': 172, 'pecsan': 173, 'perfal': 174,
    'phaino': 175, 'pibgre': 176, 'pilwoo': 177, 'pingro': 178, 'pinjay': 179,
    'pinsis': 180, 'pinwar': 181, 'plsvir': 182, 'prawar': 183, 'purfin': 184,
    'pygnut': 185, 'rebmer': 186, 'rebnut': 187, 'rebsap': 188, 'rebwoo': 189,
    'redcro': 190, 'redhea': 191, 'reevir1': 192, 'renpha': 193, 'reshaw': 194,
    'rethaw': 195, 'rewbla': 196, 'ribgul': 197, 'rinduc': 198, 'robgro': 199,
    'rocpig': 200, 'rocwre': 201, 'rthhum': 202, 'ruckin': 203, 'rudduc': 204,
    'rufgro': 205, 'rufhum': 206, 'rusbla': 207, 'sagspa1': 208, 'sagthr': 209,
    'savspa': 210, 'saypho': 211, 'scatan': 212, 'scoori': 213, 'semplo': 214,
    'semsan': 215, 'sheowl': 216, 'shshaw': 217, 'snobun': 218, 'snogoo': 219,
    'solsan': 220, 'sonspa': 221, 'sora': 222, 'sposan': 223, 'spotow': 224,
    'stejay': 225, 'swahaw': 226, 'swaspa': 227, 'swathr': 228, 'treswa': 229,
    'truswa': 230, 'tuftit': 231, 'tunswa': 232, 'veery': 233, 'vesspa': 234,
    'vigswa': 235, 'warvir': 236, 'wesblu': 237, 'wesgre': 238, 'weskin': 239,
    'wesmea': 240, 'wessan': 241, 'westan': 242, 'wewpew': 243, 'whbnut': 244,
    'whcspa': 245, 'whfibi': 246, 'whtspa': 247, 'whtswi': 248, 'wilfly': 249,
    'wilsni1': 250, 'wiltur': 251, 'winwre3': 252, 'wlswar': 253, 'wooduc': 254,
    'wooscj2': 255, 'woothr': 256, 'y00475': 257, 'yebfly': 258, 'yebsap': 259,
    'yehbla': 260, 'yelwar': 261, 'yerwar': 262, 'yetvir': 263
}

INV_BIRD_CODE = {v: k for k, v in BIRD_CODE.items()}

# Dataset

Get all TFRecords paths from GS bucket, which are already classified by fold

In [6]:
FOLDS = 5
GCS_PATH  = "gs://benayas_kaggle/birdsongs_recognition_tf/"
files_fold = { i: np.sort(np.array(tf.io.gfile.glob(GCS_PATH+"fold_"+str(i)+"*.tfrec"))) for i in range(FOLDS) }
for f,l in files_fold.items():
    print(f'Fold {f} with {len(l)} files')

Fold 0 with 95 files
Fold 1 with 93 files
Fold 2 with 95 files
Fold 3 with 95 files
Fold 4 with 98 files


Functions to convert audio to spectrogram image

In [27]:
def mono_to_color(
    X, mean=None, std=None,
    norm_max=None, norm_min=None, eps=1e-6
):
    # Stack X as [X,X,X]
    X = np.stack([X, X, X], axis=-1)

    # Standardize
    mean = mean or X.mean()
    X = X - mean
    std = std or X.std()
    Xstd = X / (std + eps)
    _min, _max = Xstd.min(), Xstd.max()
    norm_max = norm_max or _max
    norm_min = norm_min or _min
    if (_max - _min) > eps:
        # Normalize to [0, 255]
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V

def read_file(y, sr, ebird_code, period=5, img_size = 224, waveform_transforms=None, melspectrogram_parameters=None, spectrogram_transforms=None, one_hot_label=True, labeled=True):

        #sr = sr.numpy()
        print(sr)
        if waveform_transforms:
            y = waveform_transforms(y)
        else:
            len_y = len(y)
            effective_length = sr * period
            if len_y < effective_length:
                new_y = np.zeros(effective_length, dtype=y.dtype)
                start = np.random.randint(effective_length - len_y)
                new_y[start:start + len_y] = y
                y = new_y.astype(np.float32)
            elif len_y > effective_length:
                start = np.random.randint(len_y - effective_length)
                y = y[start:start + effective_length].astype(np.float32)
            else:
                y = y.astype(np.float32)

        melspec = librosa.feature.melspectrogram(y, sr=sr, **melspectrogram_parameters)
        melspec = librosa.power_to_db(melspec).astype(np.float32)
        
        librosa.display.specshow(melspec, sr=sr)

        if spectrogram_transforms:
            melspec = spectrogram_transforms(melspec)
        else:
            pass
                
        image = mono_to_color(melspec)
        height, width, _ = image.shape
        image = cv2.resize(image, (int(width * img_size / height), img_size))
        #image = np.moveaxis(image, 2, 0)  # this is for PyTorch, with shape (channels, h, w)
        image = (image / 255.0).astype(np.float32)

        # Labels in One Hot format
        if labeled:
            s = ebird_code.numpy().decode("utf-8")
            if one_hot_label:
                labels = np.zeros(len(BIRD_CODE), dtype="f")
                labels[s] = 1
            else:  # Labels in integer format
                labels = s

            return image, labels
        else:
            return image

Functions to decode tf records and to build a tf dataset

In [28]:

def read_labeled_tfrecord(ex):
    LABELED_TFREC_FORMAT = {
      'wav': tf.io.FixedLenFeature([], tf.string), # wav file
      'sr': tf.io.FixedLenFeature([], tf.int64), # sr
      'y':  tf.io.FixedLenFeature([], tf.int64), # target
    }
    example = tf.io.parse_single_example(ex, LABELED_TFREC_FORMAT)
    wav = tf.io.decode_raw(example['wav'], out_type=tf.float32)
    sr = tf.cast(example['sr'], tf.int32)
    y = tf.cast(example['y'], tf.int32)
    y = tf.one_hot(y, 264, on_value=1.0, off_value=0.0, dtype=tf.float32) # labels in one hot format
    return wav, sr, y # returns a dataset 

def cut_signal(signal, sr, waveform_transforms=None, period=5):
    if waveform_transforms:
        signal = waveform_transforms(signal)
    else:
        len_y = len(signal)
        effective_length = sr * period
        if len_y < effective_length:
            new_y = tf.zeros(effective_length, dtype=y.dtype)
            start = tf.random.uniform(effective_length - len_y)
            new_y[start:start + len_y] = y
            y = tf.cast(new_y, tf.float32)
        elif len_y > effective_length:
            start = tf.random.uniform(len_y - effective_length)
            y = tf.cast(y[start:start + effective_length], tf.float32)
        else:
            y = tf.cast(y, tf.float32)

    return signal

def tf_log10(x):
    numerator = tf.math.log(x)
    denominator = tf.math.log(tf.constant(10, dtype=numerator.dtype))
    return numerator / denominator

def power_to_db(magnitude, amin=1e-16, top_db=80.0):
    ref_value = tf.reduce_max(magnitude)
    log_spec = 10.0 * tf_log10(tf.maximum(amin, magnitude))
    log_spec -= 10.0 * tf_log10(tf.maximum(amin, ref_value))
    log_spec = tf.maximum(log_spec, tf.reduce_max(log_spec) - top_db)

    return log_spec

def spectrogram(signals, sr, mel_params):
    spectrograms = tf.signal.stft(waveforms,
                                  frame_length=mel_params['fft_size'],
                                  frame_step=mel_params['hop_size'],
                                  pad_end=False)

    magnitude_spectrograms = tf.abs(spectrograms)

    mel_filterbank = tf.signal.linear_to_mel_weight_matrix(
                      num_mel_bins=mel_params['n_mels'],
                      num_spectrogram_bins=mel_params['fft_size'] // 2 + 1,
                      sample_rate=sr,
                      lower_edge_hertz=mel_params['fmin'],
                      upper_edge_hertz=mel_params['fmax'])

    mel_spectrograms = tf.matmul(tf.square(magnitude_spectrograms), mel_filterbank)

    log_mel_spectrograms = power_to_db(mel_spectrograms)

    # add channel dimension
    log_mel_spectrograms = tf.expand_dims(log_mel_spectrograms, 3)

    return log_mel_spectrograms



def load_dataset(filenames, mel_params, batch_size=32, labeled=True, shuffle=False, repeat=False, waveform_transforms=None, n_classes=264):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.cache()

    if repeat:
        dataset = dataset.repeat()
    
    if shuffle: 
        dataset = dataset.shuffle(32*REPLICAS)
        opt = tf.data.Options()
        opt.experimental_deterministic = False # disable order, increase speed
        dataset = dataset.with_options(opt)

    dataset = dataset.map(read_labeled_tfrecord, num_parallel_calls=AUTO) # Decode TF Records

    # Generate spectrogram
    dataset = dataset.map(lambda wav, sr, label: (cut_signal(wav, sr, waveform_transforms), sr, label), num_parallel_calls=AUTO)  # fourier short term transform
    dataset = dataset.map(lambda wav, sr, label: (spectrogram(wav, sr, mel_params), sr, label), num_parallel_calls=AUTO)  # fourier short term transform

    #dataset = dataset.map(lambda wav, sr, label: read_file(tf.reshape(wav,[-1]), sr, label, period=5, 
    #                                                       img_size = 224, 
    #                                                      waveform_transforms=None, 
    #                                                       melspectrogram_parameters=melspectrogram_parameters, 
    #                                                       spectrogram_transforms=None, 
    #                                                       one_hot_label=True, 
    #                                                       labeled=labeled))

    
    dataset = dataset.batch(batch_size*REPLICAS)
    dataset = dataset.prefetch(AUTO)
    return dataset

def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(f[:f.rfind('.')].split('_')[-1]) for f in filenames]
    return np.sum(n)

def get_fold(fold, files_fold):
    train_files = np.concatenate([v for k,v in files_fold.items() if k != fold])
    val_files = files_fold[fold]
    return train_files, val_files

# Model

In [14]:
# Function to be called
def get_model(model, n_classes):
    if model == 0:
        return get_model0(0, n_classes)

EFNS = [efn.EfficientNetB0, efn.EfficientNetB1, efn.EfficientNetB2, efn.EfficientNetB3,
        efn.EfficientNetB4, efn.EfficientNetB5, efn.EfficientNetB6, efn.EfficientNetB7]

# Model based on efficient net B0
def get_model0(b, n_classes, shape=(224,547,3)):

    # inputs
    inp = tf.keras.layers.Input(shape=shape)
    base = EFNS[b](input_shape=shape, weights='imagenet', include_top=False) # Efficientnet
    x = base(inp)

    # # use the same head as the baseline notebook.
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.Dropout(rate=0.2)(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.Dropout(rate=0.2)(x)
    x = tf.keras.layers.Dense(n_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs=inp, outputs=x)

    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    loss = tf.keras.losses.CategoricalCrossentropy() 
    model.compile(optimizer=opt, loss=loss)
    
    return model

def get_lr_callback(batch_size=8, lr_min = 0.000001, lr_ramp_ep = 5, lr_sus_ep = 0, lr_decay=0.8):
    lr_start   = 0.000005
    lr_max     = 0.00000125 * REPLICAS * batch_size
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
            
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max
            
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
            
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=False)
    return lr_callback

# Experiments

In [10]:
N_EXPERIMENTS = 1  # Normally not more than one run per commit
FOLD = 0 # Each run should cover a single fold

# DATASET PARAMS
IMG_SIZE = [224] * N_EXPERIMENTS
MELSPECTOGRAM_PARAMS = [{'n_mels':128, 'fmin':20, 'fmax':16000, 'hop_size':512, 'fft_size':1024}] * N_EXPERIMENTS

# DATALOADER PARAMS
BS_TRAIN = [64] * N_EXPERIMENTS
BS_VAL = [64] * N_EXPERIMENTS

# MODEL PARAMS
MODEL = [0] * N_EXPERIMENTS

# LR SCHEDULER
LR_RAMP_EP = [5] * N_EXPERIMENTS
LR_DECAY = [0.8] * N_EXPERIMENTS

# GLOBAL PARAMETERS
EPOCHS=40
N_CLASSES = 264
DISPLAY_PLOT=True
VERBOSE = 1

# Training Loop

In [29]:
i = 0
_, files_val = get_fold(i, files_fold)
ds_check = load_dataset(files_val[:1], batch_size=1, labeled=True, shuffle=False, repeat=False, melspectrogram_parameters=MELSPECTOGRAM_PARAMS[i])
for wav, sr, label in ds_check.take(1):
    print(wav.shape)


Tensor("args_1:0", shape=(), dtype=int32)


TypeError: ignored

In [22]:
for i in range(N_EXPERIMENTS):
    print(f'********** EXPERIMENT {i} **********')
    print(f'***** img size {IMG_SIZE[i]} *****')
    print(f'***** bs train {BS_TRAIN[i]*REPLICAS} *****')
    print(f'***** bs val {BS_VAL[i]*REPLICAS} *****')
    print(f'***** model {MODEL[i]} *****')
    print(f'***** ramp epochs {LR_RAMP_EP[i]} *****')
    print(f'***** decay epochs {LR_DECAY[i]} *****')
    print(f'**********************************\n')

    # DISPLAY FOLD INFO
    if DEVICE=='TPU':
        if tpu: tf.tpu.experimental.initialize_tpu_system(tpu)
    
    # CREATE TRAIN AND VALIDATION DATASETS
    files_train, files_val = get_fold(i, files_fold)
    
    # BUILD MODEL
    print('Building model...')
    K.clear_session()
    with strategy.scope():
        model = get_model(MODEL[i], N_CLASSES)

    # SAVE BEST MODEL EACH FOLD
    sv = tf.keras.callbacks.ModelCheckpoint('/content/drive/My Drive/Kaggle/birdsongs/'+'fold-%i.h5'%i, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min', save_freq='epoch')
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
   
    # TRAIN
    print('Training...')
    history = model.fit(
        load_dataset(files_train, batch_size=BS_TRAIN[i]*REPLICAS, labeled=True, shuffle=True, repeat=True, melspectrogram_parameters=MELSPECTOGRAM_PARAMS[i]),
        epochs = EPOCHS[i], 
        callbacks = [es,sv,get_lr_callback(BS_TRAIN[i]*REPLICAS)], 
        steps_per_epoch = count_data_items(files_train)/BS_TRAIN[i]//REPLICAS,
        validation_data = load_dataset(files_val, batch_size=BS_VAL[i]*REPLICAS, labeled=True, shuffle=False, repeat=False, melspectrogram_parameters=MELSPECTOGRAM_PARAMS[i]),
        verbose = VERBOSE
    )

    # PLOT TRAINING
    if DISPLAY_PLOT:
        plt.figure(figsize=(15,5))
        plt.plot(np.arange(len(history.history)), history.history['train'],'-o',label='Train Loss',color='#ff7f0e')
        plt.plot(np.arange(len(history.history)), history.history['val'],'-o',label='Val Loss',color='#1f77b4')
        x = np.argmin( history['val'] ); y = np.min( history['val'] )
        xdist = plt.xlim()[1] - plt.xlim()[0]; ydist = plt.ylim()[1] - plt.ylim()[0]
        plt.text(x-0.03*xdist,y-0.13*ydist,'min loss\n%.2f'%y,size=14)
        plt.ylabel('Loss',size=14); plt.xlabel('Epoch',size=14)
        plt.legend(loc=2)
        
        plt2 = plt.gca().twinx()
        plt2.plot(np.arange(len(history.history)),history.history['lr'],'-o',label='LR',color='#2ca02c')
        plt.ylabel('LR',size=14)
        
        plt.title('Experiment %i'%i,size=18)
        plt.legend(loc=3)
        plt.show()
    
    print('\n')

********** EXPERIMENT 0 **********
***** img size 224 *****
***** bs train 512 *****
***** bs val 512 *****
***** model 0 *****
***** ramp epochs 5 *****
***** decay epochs 0.8 *****
**********************************



INFO:tensorflow:Initializing the TPU system: grpc://10.91.190.234:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.91.190.234:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


Building model...
Training...


ValueError: ignored

In [33]:
sr = tf.constant(32000)
sr

y = tf.random.uniform((100,))
y

<tf.Tensor: shape=(100,), dtype=float32, numpy=
array([0.27678192, 0.34663165, 0.9547851 , 0.7169672 , 0.41563833,
       0.1360488 , 0.8720342 , 0.2773627 , 0.54786897, 0.10202849,
       0.40615594, 0.4824822 , 0.7166357 , 0.8600658 , 0.82450974,
       0.79715073, 0.8275049 , 0.83120406, 0.42319465, 0.19195008,
       0.28318477, 0.08628213, 0.4876535 , 0.97954893, 0.49209118,
       0.81923306, 0.81524754, 0.46429896, 0.23481989, 0.6655687 ,
       0.70842135, 0.04478478, 0.9397402 , 0.91016173, 0.72685874,
       0.7116989 , 0.5615134 , 0.13452506, 0.787449  , 0.6652119 ,
       0.6917943 , 0.7350787 , 0.13160408, 0.09890378, 0.6431197 ,
       0.05490696, 0.26530802, 0.5927067 , 0.5184516 , 0.3629428 ,
       0.06924224, 0.17468858, 0.0152564 , 0.5905843 , 0.9495995 ,
       0.86046493, 0.1052829 , 0.75908923, 0.09936297, 0.06437576,
       0.47076094, 0.8954743 , 0.04200876, 0.1520083 , 0.03838038,
       0.21169806, 0.4053259 , 0.6009524 , 0.71824515, 0.94588125,
       0.01138